Простой пример: Запуск Компьютера
Представим, что включение компьютера — это сложный процесс, включающий проверку BIOS, загрузку ОС, запуск сервисов. Фасад может скрыть эти шаги за одним простым методом start().

In [1]:
# --- Subsystem Classes (Классы Подсистемы) ---

class CPU:
    """Часть подсистемы: Процессор."""
    def freeze(self):
        print("CPU: Freezing processor.")
    def jump(self, position):
        print(f"CPU: Jumping to address {position}.")
    def execute(self):
        print("CPU: Executing instructions.")

class Memory:
    """Часть подсистемы: Память."""
    def load(self, position, data):
        print(f"Memory: Loading data '{data}' to address {position}.")

class HardDrive:
    """Часть подсистемы: Жесткий диск."""
    def read(self, lba, size):
        print(f"HardDrive: Reading {size} sectors from LBA {lba}.")
        return f"Data from sector {lba}" # Имитация данных

# --- Facade (Фасад) ---

class ComputerFacade:
    """
    Фасад, предоставляющий простой интерфейс для запуска компьютера.
    Он скрывает сложное взаимодействие между CPU, Memory и HardDrive.
    """
    def __init__(self):
        print("ComputerFacade: Initializing components...")
        self._cpu = CPU()
        self._memory = Memory()
        self._hard_drive = HardDrive()
        self._boot_address = 0x0000
        self._boot_sector = 0

    def start(self):
        """Запускает компьютер одним вызовом."""
        print("\nComputerFacade: Starting computer...")
        self._cpu.freeze()
        self._memory.load(self._boot_address, self._hard_drive.read(self._boot_sector, 1))
        self._cpu.jump(self._boot_address)
        self._cpu.execute()
        print("ComputerFacade: Computer started successfully!")

# --- Client Code (Клиентский Код) ---

if __name__ == "__main__":
    # Клиенту не нужно знать о CPU, Memory, HardDrive.
    # Он просто использует Фасад.
    computer = ComputerFacade()
    computer.start() # Один простой вызов

# Вывод:
# ComputerFacade: Initializing components...
#
# ComputerFacade: Starting computer...
# CPU: Freezing processor.
# HardDrive: Reading 1 sectors from LBA 0.
# Memory: Loading data 'Data from sector 0' to address 0.
# CPU: Jumping to address 0.
# CPU: Executing instructions.
# ComputerFacade: Computer started successfully!

ComputerFacade: Initializing components...

ComputerFacade: Starting computer...
CPU: Freezing processor.
HardDrive: Reading 1 sectors from LBA 0.
Memory: Loading data 'Data from sector 0' to address 0.
CPU: Jumping to address 0.
CPU: Executing instructions.
ComputerFacade: Computer started successfully!


Сложный пример: Система Онлайн-заказа
Рассмотрим интернет-магазин. Процесс оформления заказа включает проверку наличия товара, обработку платежа и организацию доставки. Фасад OrderFacade упрощает этот процесс для клиента.

In [2]:
import random
import string

# --- Subsystem Classes (Классы Подсистемы) ---

class InventorySystem:
    """Подсистема: Проверка наличия товара."""
    def __init__(self):
        # Имитация базы данных склада
        self._stock = {"item123": 5, "item456": 0, "item789": 10}

    def check_stock(self, item_id: str, quantity: int) -> bool:
        print(f"Inventory: Checking stock for {quantity} of '{item_id}'...")
        available = self._stock.get(item_id, 0)
        if available >= quantity:
            print(f"Inventory: Stock available for '{item_id}'.")
            return True
        else:
            print(f"Inventory: Insufficient stock for '{item_id}' (available: {available}).")
            return False

    def reserve_item(self, item_id: str, quantity: int) -> bool:
        """Резервирует товар на складе (упрощено)."""
        if self.check_stock(item_id, quantity):
            self._stock[item_id] -= quantity
            print(f"Inventory: Reserved {quantity} of '{item_id}'. New stock: {self._stock[item_id]}")
            return True
        return False

class PaymentSystem:
    """Подсистема: Обработка платежей."""
    def process_payment(self, card_number: str, amount: float) -> bool:
        print(f"Payment: Attempting to charge ${amount:.2f} from card ending in ...{card_number[-4:]}")
        # Имитация проверки и обработки платежа
        if len(card_number) == 16 and random.choice([True, True, False]): # 2/3 успеха
            print(f"Payment: Successfully charged ${amount:.2f}.")
            return True
        else:
            print("Payment: Payment failed (invalid card or insufficient funds).")
            return False

class ShippingSystem:
    """Подсистема: Организация доставки."""
    def schedule_shipping(self, order_id: str, address: str) -> str | None:
        print(f"Shipping: Scheduling delivery for order '{order_id}' to address '{address}'.")
        # Имитация планирования и получения трекинг-номера
        if address:
            tracking_number = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
            print(f"Shipping: Delivery scheduled. Tracking number: {tracking_number}")
            return tracking_number
        else:
            print("Shipping: Invalid address provided.")
            return None

# --- Facade (Фасад) ---

class OrderFacade:
    """
    Фасад для упрощения процесса оформления заказа.
    Координирует работу систем Inventory, Payment и Shipping.
    """
    def __init__(self):
        print("OrderFacade: Initializing subsystems...")
        self._inventory = InventorySystem()
        self._payment = PaymentSystem()
        self._shipping = ShippingSystem()

    def place_order(self, item_id: str, quantity: int, price_per_item: float,
                      card_number: str, shipping_address: str) -> str:
        """
        Размещает заказ, выполняя все необходимые шаги через подсистемы.
        Возвращает сообщение о статусе заказа.
        """
        print(f"\nOrderFacade: Attempting to place order for {quantity} x '{item_id}'...")
        order_status = "Order Failed"
        tracking_number = None

        # 1. Проверка наличия и резервирование товара
        if not self._inventory.reserve_item(item_id, quantity):
            return f"{order_status}: Item '{item_id}' is out of stock or quantity not available."

        # 2. Обработка платежа
        total_amount = quantity * price_per_item
        if not self._payment.process_payment(card_number, total_amount):
            # В реальной системе нужно отменить резервирование товара
            print("OrderFacade: Payment failed, cancelling inventory reservation (simulated).")
            # self._inventory.cancel_reservation(item_id, quantity) # Добавить метод отмены
            return f"{order_status}: Payment processing failed."

        # 3. Генерация ID заказа (упрощено)
        order_id = ''.join(random.choices(string.digits, k=8))
        print(f"OrderFacade: Payment successful. Order ID: {order_id}")

        # 4. Организация доставки
        tracking_number = self._shipping.schedule_shipping(order_id, shipping_address)
        if not tracking_number:
            # В реальной системе нужно отменить платеж и резервирование
            print("OrderFacade: Shipping scheduling failed, cancelling payment and reservation (simulated).")
            # self._payment.refund(...)
            # self._inventory.cancel_reservation(...)
            return f"{order_status}: Shipping address is invalid."

        # 5. Успешное завершение
        order_status = "Order Placed Successfully"
        print(f"OrderFacade: Order completed for '{item_id}'.")
        return f"{order_status}! Order ID: {order_id}, Tracking: {tracking_number}"

# --- Client Code (Клиентский Код) ---

if __name__ == "__main__":
    order_system = OrderFacade()

    print("\n--- Client placing first order ---")
    result1 = order_system.place_order(
        item_id="item123",
        quantity=2,
        price_per_item=50.0,
        card_number="1111222233334444",
        shipping_address="123 Main St, Anytown"
    )
    print(f"Client received status: {result1}")

    print("\n--- Client placing second order (item out of stock) ---")
    result2 = order_system.place_order(
        item_id="item456",
        quantity=1,
        price_per_item=100.0,
        card_number="5555666677778888",
        shipping_address="456 Oak Ave, Sometown"
    )
    print(f"Client received status: {result2}")

    print("\n--- Client placing third order (potential payment fail) ---")
    result3 = order_system.place_order(
        item_id="item789",
        quantity=1,
        price_per_item=25.0,
        card_number="9999000011112222", # Может пройти, может нет
        shipping_address="789 Pine Ln, Otherville"
    )
    print(f"Client received status: {result3}")


# Примерный вывод (результат платежа и трекинг-номер могут меняться):
# OrderFacade: Initializing subsystems...
#
# --- Client placing first order ---
#
# OrderFacade: Attempting to place order for 2 x 'item123'...
# Inventory: Checking stock for 2 of 'item123'...
# Inventory: Stock available for 'item123'.
# Inventory: Reserved 2 of 'item123'. New stock: 3
# Payment: Attempting to charge $100.00 from card ending in ...4444
# Payment: Successfully charged $100.00.
# OrderFacade: Payment successful. Order ID: 78510349
# Shipping: Scheduling delivery for order '78510349' to address '123 Main St, Anytown'.
# Shipping: Delivery scheduled. Tracking number: J8Y3L0GZQW
# OrderFacade: Order completed for 'item123'.
# Client received status: Order Placed Successfully! Order ID: 78510349, Tracking: J8Y3L0GZQW
#
# --- Client placing second order (item out of stock) ---
#
# OrderFacade: Attempting to place order for 1 x 'item456'...
# Inventory: Checking stock for 1 of 'item456'...
# Inventory: Insufficient stock for 'item456' (available: 0).
# Client received status: Order Failed: Item 'item456' is out of stock or quantity not available.
#
# --- Client placing third order (potential payment fail) ---
#
# OrderFacade: Attempting to place order for 1 x 'item789'...
# Inventory: Checking stock for 1 of 'item789'...
# Inventory: Stock available for 'item789'.
# Inventory: Reserved 1 of 'item789'. New stock: 9
# Payment: Attempting to charge $25.00 from card ending in ...2222
# Payment: Payment failed (invalid card or insufficient funds).
# OrderFacade: Payment failed, cancelling inventory reservation (simulated).
# Client received status: Order Failed: Payment processing failed.

OrderFacade: Initializing subsystems...

--- Client placing first order ---

OrderFacade: Attempting to place order for 2 x 'item123'...
Inventory: Checking stock for 2 of 'item123'...
Inventory: Stock available for 'item123'.
Inventory: Reserved 2 of 'item123'. New stock: 3
Payment: Attempting to charge $100.00 from card ending in ...4444
Payment: Successfully charged $100.00.
OrderFacade: Payment successful. Order ID: 73688180
Shipping: Scheduling delivery for order '73688180' to address '123 Main St, Anytown'.
Shipping: Delivery scheduled. Tracking number: 0AH7XLEWQY
OrderFacade: Order completed for 'item123'.
Client received status: Order Placed Successfully! Order ID: 73688180, Tracking: 0AH7XLEWQY

--- Client placing second order (item out of stock) ---

OrderFacade: Attempting to place order for 1 x 'item456'...
Inventory: Checking stock for 1 of 'item456'...
Inventory: Insufficient stock for 'item456' (available: 0).
Client received status: Order Failed: Item 'item456' is out o